In [1]:

import pandas as pd
#import numpy as np
import string
import re
#import glob
#import os
#import operator
import nltk
print('The nltk version is {}.'.format(nltk.__version__))
from nltk.corpus import stopwords
from stemming.porter2 import stem

The nltk version is 3.2.5.


In [3]:
df = pd.read_csv('C:/Users/Steven Ma/Desktop/ASU - Business Analytics/Applied Project/Text Data/Training_Test.csv', sep=',')

pos_tweets = []
neg_tweets = []

for index, row in df.iterrows():
    if row['Sentiment'] == "Pos":
        pos_tuple = tuple([row['SentimentText'], row['Sentiment']])
        pos_tweets.append(pos_tuple)

for index, row in df.iterrows():
    if row['Sentiment'] == "Neg":
        neg_tuple = tuple([row['SentimentText'], row['Sentiment']])
        neg_tweets.append(neg_tuple)

In [4]:
# Text processing
tweets = []
regex = re.compile('[%s]' % re.escape(string.punctuation))
sw = stopwords.words("english")

for (words, sentiment) in pos_tweets + neg_tweets:
    words_removed = regex.sub('', words)
    words_lowered = [e.lower() for e in words_removed.split()]
    words_nonstopped = [w for w in words_lowered if w not in sw]
    words_stemmed = [stem(txt) for txt in words_nonstopped]
    tweets.append((words_stemmed, sentiment))
#print(tweets)

In [5]:
# User defined function to extract list of Word Features
def get_words_in_tweets(tweets):
    all_words = []
    for (words, sentiment) in tweets:
      all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    WordList = nltk.FreqDist(wordlist)
    print("=====Feature Expression=====")
    print(WordList)
    print("====Top 10 Most Common=====")
    print(WordList.most_common(10))
    print("============================")
    word_features = list(WordList.keys())
    return word_features

In [6]:
# Extract list of Word Features
word_features = get_word_features(get_words_in_tweets(tweets))
#print("Word Features :", word_features)

=====Feature Expression=====
<FreqDist with 543 samples and 769 outcomes>
====Top 10 Most Common=====
[('day', 10), ('good', 9), ('haha', 7), ('im', 7), ('miss', 7), ('go', 7), ('back', 7), ('home', 6), ('thank', 5), ('like', 5)]


In [7]:
# Feature Extractor
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['%s' % word] = (word in document_words)
    return features

Extract_Features = extract_features(word_features)
print("Extract_Features :", Extract_Features)

Extract_Features : {'redinkdiari': True, 'realli': True, 'finger': True, 'point': True, 'big': True, 'chanc': True, 'one': True, 'public': True, 'well': True, 'known': True, 'dont': True, 'need': True, 'insid': True, 'sourc': True, 'clatsyrc': True, 'thank': True, 'post': True, 'alway': True, 'like': True, 'feedback': True, 'realiz': True, 'sound': True, 'lot': True, 'kati': True, 'perri': True, 'haha': True, '2niteboy': True, 'idea': True, 'get': True, 'result': True, 'week': True, 'whaa': True, 'intens': True, 'type247': True, 'true': True, 'boss': True, 'let': True, 'wear': True, 'write': True, 'pant': True, 'sheep': True, 'slipper': True, 'lovelylondon': True, 'pretti': True, 'good': True, 'littleheadcas': True, 'use': True, 'xcritic': True, 'hope': True, 'steer': True, 'right': True, 'direct': True, 'watch': True, 'citi': True, 'god': True, 'yesterday': True, 'still': True, 'noodl': True, 'cup': True, 'church': True, 'look': True, 'men': True, 'choir': True, 'quotso': True, 'glad'

In [8]:
# Training Set

training_set = nltk.classify.util.apply_features(extract_features, tweets)
print("Training Set :", training_set)

Training Set : [({'redinkdiari': True, 'realli': True, 'finger': True, 'point': True, 'big': True, 'chanc': True, 'one': True, 'public': True, 'well': True, 'known': True, 'dont': True, 'need': True, 'insid': True, 'sourc': True, 'clatsyrc': False, 'thank': False, 'post': False, 'alway': False, 'like': False, 'feedback': False, 'realiz': False, 'sound': False, 'lot': False, 'kati': False, 'perri': False, 'haha': False, '2niteboy': False, 'idea': False, 'get': False, 'result': False, 'week': False, 'whaa': False, 'intens': False, 'type247': False, 'true': False, 'boss': False, 'let': False, 'wear': False, 'write': False, 'pant': False, 'sheep': False, 'slipper': False, 'lovelylondon': False, 'pretti': False, 'good': False, 'littleheadcas': False, 'use': False, 'xcritic': False, 'hope': False, 'steer': False, 'right': False, 'direct': False, 'watch': False, 'citi': False, 'god': False, 'yesterday': False, 'still': False, 'noodl': False, 'cup': False, 'church': False, 'look': False, 'men'

In [9]:
# Training Classifier

classifier = nltk.NaiveBayesClassifier.train(training_set)